In [13]:
import json
import requests

query = "best res"

cursor_pos = len(query)
url_friendly_query = query.replace(" ", "%20")

url = f"https://www.google.com/complete/search?q={url_friendly_query}%20reddit&cp={cursor_pos}&client=gws-wiz&xssi=t&hl=en&authuser=0&dpr=2"

res = requests.get(url).text[5:]
parsed = json.loads(res)[0]

In [14]:
suggestions = [element[0].replace("reddit", "").replace(" </b>", "</b>") for element in parsed]

In [15]:
suggestions

['best res<b>ume builder</b>',
 'best res<b>ident evil game</b>',
 'best res<b>ume template</b>',
 'best res<b>istance bands</b>',
 'best res<b>taurants nyc</b>',
 'best res<b>ume writing service</b>',
 'best res<b>taurants chicago</b>',
 'best res<b>taurants montreal</b>',
 'best res<b>p canada</b>',
 'best res<b>taurants in rome</b>']